마르코프체인은 엄격하게 학습모형은 아니다. 들어오는 문장을 형태소분석한 다음에 카운트, json으로 정리해서 가지고 있다. 그것을 토대로 해서 지금 들어온 단어 다음에는 뭐가 올지 말해준다.

In [1]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt
import urllib.request
import os, re, json, random

In [6]:
# 마르코프 체인 딕셔너리 만들기 --- (※1)
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic
# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1
# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic" 
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".": break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 띄어쓰기
    params = urllib.parse.urlencode({
        "_callback": "",
        "q": ret
    })
#     # 네이버 맞춤법 검사기를 사용합니다.
#     data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/dcontent/spellchecker.nhn?" + params)
#     data = data.read().decode("utf-8")[1:-2]
#     data = json.loads(data)
#     data = data["message"]["result"]["html"]
#     data = soup = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return ret

def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))
# 문장 읽어 들이기 --- (※4)
toji_file = "toji.txt"
dict_file = "markov-toji.json"
if not os.path.exists(dict_file):
    # 토지 텍스트 파일 읽어 들이기
    fp = codecs.open("BEXX0014.txt", "r", encoding="utf-16")
    soup = BeautifulSoup(fp, "html.parser")
    body = soup.select_one("text > body")
    text = body.getText()
    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    # 형태소 분석
    twitter = Okt()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file,"r"))    

# 다음 맞춤법 검사
def spell(s):
    import requests
    data={}
    data["sentence"]=s.encode("utf-8")
    r = requests.post("https://alldic.daum.net/grammar_checker.do", data=data)
    soup = BeautifulSoup(r.text,'html.parser')
    result=[]
    for text in soup.find_all('a'):
        if not text.get('data-error-output') == None:
            a=text.get('data-error-output')
            result.append(a)
    return(result)
    
# 문장 만들기 --- (※6)
for i in range(3):
    s = make_sentence(dic)
    print(s)
    print(spell(s))
    print("---")

어젯밤에도작은아버님말씀을하시다가

어차피만주에있을적에도손질한무명옷발이고운흰베를포근포근다듬어서지은홑겹두루마기를벗어걸어놓고무료하게마루에올라가거라점심묵자
호야네는임의롭게핀잔주듯말했다.
['어젯밤에도 작은아버님 말씀을 하시다가', '어차피 만주에 있을 적에도 손질한 무명 옷발이고 운 흰 베를 포근포근 다듬어서 지은 홑겹 두루마기를벗어걸어놓고무료하게 마루에 올라가 거라 점심 묵자', '호야네는 임의롭게 핀잔주듯 말했다.']
---
자질구레한이것만묵고앉아있겄소
마음속으로홍이한테간다면서떠났다.
['자질구레한 이것만 묵고 앉아 있겠소', '마음속으로 홍 이한 테간 다면서 떠났다.']
---
가사친지를통해서말치의보리떡이란보리서말했다.
['가사 친지를 통해서말치의보리떡이란보리 서말 했다.']
---
